# Notebook Overview

This notebook demonstrates a workflow for transforming tabular data extracted from PDFs into a searchable vector store for downstream retrieval tasks.

## Main Steps

1. **Transform Table Rows to JSON**
   - Reads all CSV files matching `output_table*.csv`.
   - Cleans table headers for consistency.
   - Converts each row to a structured JSON object containing the table name, a text representation of the row, and metadata (page number, column names).
   
```json
{
    
    "text": "<column1>: <value1> | <column2>: <value2> | ...",
    "metadata": {
         "table_name": "<name>",
        "page": <page_number>,
        "columns": [<column_names>]
    }
}
```
   - Saves the resulting JSON objects to new files.

2. **Build a Vector Store**
   - Loads the generated JSON files.
   - Converts each JSON object into a `Document` for use with vector databases.
   - Uses a sentence transformer model to create embeddings.
   - Stores the embeddings in a FAISS vector store for efficient similarity search.

3. **Query the Vector Store**
   - Demonstrates how to retrieve the most relevant table rows for a given query using the vector store.

In [35]:
import pandas as pd
import glob
import re
import json

file_names = glob.glob("../data/output_table*.csv")

def clean_header(df):
    # replace special chars and spaces with underscores in headers
    df.columns = df.columns.str.strip().str.lower().str.replace(r'[^A-Za-z0-9]+', '_', regex=True).str.strip('_')
    df.columns = df.columns.str.replace('_none', '')
    return df

def chunk(row, table_name, page_num):
    text = " | ".join(f"{col}: {val}" for col, val in row.items() if pd.notna(val))
    return {
        "text": text,
        "metadata": {
            "table_name": table_name,
            "page": page_num,
            "columns": list(row.index)
        }
    }
for file in file_names:
    df = pd.read_csv(file)
    df = clean_header(df)
    table_name = file.split("\\")[-1].replace(".csv", "")
    match = re.search(r'table(\d+)', file)
    page_num = int(match.group(1)) if match else -1
    chunks = [chunk(row, table_name, page_num) for _, row in df.iterrows()]
    json.dump(chunks, open(file.replace(".csv", ".json"), "w"), indent=4)

In [38]:
import json
import glob 
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings import SentenceTransformerEmbeddings



file_names = glob.glob("../data/output_table*.json")
documents = []
for file in file_names:
    with open(file, "r") as f:
        chunks = json.load(f)
    
    for chunk in chunks:
        document = Document(
            page_content=chunk["text"],
            metadata=chunk["metadata"]
        )
        documents.append(document)
    
# Create embedding model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local("../data/faiss_index")   


In [43]:
def retrieve(query, k=3):
    """
    Retrieve top-k relevant documents from FAISS vector store.
    """
    results = vector_store.similarity_search(query, k=k)
    for i, doc in enumerate(results):
        print(f"\nResult {i+1}:")
        print("Text:", doc.page_content)
        print("Metadata:", doc.metadata)

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.load_local("../data/faiss_index", embeddings, allow_dangerous_deserialization=True)

query = "Arsenic TRV value Inhalation"
retrieve(query, k=1)


Result 1:
Text: substance: Arsenic | type_of_trv: Inhalation UR | trv_value: 6.4E+00
(mg/m3)–¹ | study_details: Study Type: epidemiological
(occupational)
Species: humans
Mode of Exposure: inhalation
Exposure Concentrations: N/A
Duration: chronic
Uncertainty Factors: N/A | threshold_non_threshold_endpoint: TC (5% tumourigenic
05
concentration) =
7.83 µg/m³ | trv_derivation_method: Relative risk model
Inhalation UR =
0.05/TC
05
where 0.05 = 5% extra
cancer risk | critical_effect_s: Cancer
(lung) | source: EC and HC, 1993a
(based on
Higgins et al.,
1986)
Metadata: {'table_name': 'output_table24', 'page': 24, 'columns': ['substance', 'type_of_trv', 'trv_value', 'study_details', 'threshold_non_threshold_endpoint', 'trv_derivation_method', 'critical_effect_s', 'carcinogenicity_classification', 'source']}
